In [1]:
import math
import pickle
import numpy as np
import pandas as pd
import argparse
import torch
from torch import nn
import torch.nn.functional as F
from train_functions import *
from feature_functions import load_pickle, dump_pickle

from torch.autograd import Variable
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import warnings
import random



parser = argparse.ArgumentParser(description='Inputs: --train_path: train data path; --test_path: test data path;\
                        --lr: learning rate;--batch: batch size; --lr_decay:multiplicative factor of learning rate decay. Default: 0.5;\
                        --decay_interval: period of learning rate decay; --num_epoch: the number of epochs;\
                        --param_dict_pkl: the path to parameters; --shuffle_T: shuffle temperature feature if True. Default=False')
    
parser.add_argument('--train_path', default='../data/')
parser.add_argument('--test_path', default='../data/')
parser.add_argument('--lr', default = 0.001, type=float )
parser.add_argument('--batch', default = 1 , type=int )
parser.add_argument('--lr_decay', default = 0.5, type=float )
parser.add_argument('--decay_interval', default = 10, type=int )
parser.add_argument('--num_epoch', default = 5, type=int )
parser.add_argument('--seed', default = 42, type=int )
parser.add_argument('--param_dict_pkl', default = '../data/hyparams/param_2.pkl')
parser.add_argument('--shuffle_T', default = 'False', choices=['False','True'], type = str )
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


train_path, test_path, lr, batch_size, lr_decay, decay_interval, param_dict_pkl = \
        str(args.train_path), str(args.test_path), float(args.lr), int(args.batch), \
        float(args.lr_decay), int(args.decay_interval) , str( args.param_dict_pkl )

print('Loading train data from %s .' % train_path)
if not ( os.path.isdir(train_path) ):
    raise SystemExit('Directory %s does not exist!' % train_path )
    
print('Loading test data from %s .' % test_path)
if not ( os.path.isdir(test_path) ):
    raise SystemExit('Directory %s does not exist!' % test_path )
    
print('Loading parameters from %s .' % param_dict_pkl)
if not ( os.path.exists(param_dict_pkl) ):
    raise SystemExit('File %s does not exist!' % param_dict_pkl )
    
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.manual_seed_all(args.seed)

print(args)

param_dict = load_pickle(param_dict_pkl)
comp_dim, prot_dim, gat_dim, num_head, dropout, alpha, window, layer_cnn, latent_dim, layer_out = \
        param_dict['comp_dim'], param_dict['prot_dim'],param_dict['gat_dim'],param_dict['num_head'],\
        param_dict['dropout'], param_dict['alpha'], param_dict['window'], param_dict['layer_cnn'], \
        param_dict['latent_dim'], param_dict['layer_out']

print(param_dict)
warnings.filterwarnings("ignore", message="Setting attributes on ParameterList is not supported.")

/opt/anaconda3/envs/Uni_test/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading train data from ../data/ .
Loading test data from ../data/ .
Loading parameters from ../data/hyparams/param_2.pkl .
Namespace(batch=1, decay_interval=10, lr=0.001, lr_decay=0.5, num_epoch=5, param_dict_pkl='../data/hyparams/param_2.pkl', seed=42, shuffle_T='False', test_path='../data/', train_path='../data/')
{'comp_dim': 80, 'prot_dim': 80, 'gat_dim': 50, 'num_head': 3, 'dropout': 0.1, 'alpha': 0.1, 'window': 5, 'layer_cnn': 4, 'latent_dim': 40, 'layer_out': 4}


In [2]:
def load_esm_data( path, has_label, type ):
    '''
    Load features generated by /code/gen_features.py.
    '''
    compounds = np.array( load_pickle( os.path.join(path, '{}_features_esm/compounds.pkl'.format(type)) ), dtype=object )
    adjacencies = np.array( load_pickle( os.path.join(path, '{}_features_esm/adjacencies.pkl'.format(type)) ), dtype=object )
    fps =  np.array( load_pickle( os.path.join(path, '{}_features_esm/fps.pkl'.format(type)) ), dtype=object )
    proteins = load_pickle( os.path.join(path, '{}_features_esm/proteins.pkl'.format(type)) )
    inv_Temp = np.array( load_pickle( os.path.join(path,  '{}_features_esm/inv_Temp.pkl'.format(type)) ) , dtype=object)
    Temp = np.array( load_pickle( os.path.join(path,  '{}_features_esm/Temp.pkl'.format(type)) ) , dtype=object)
    if has_label:
        targets = np.array( load_pickle( os.path.join(path,  '{}_features_esm/log10_kcat.pkl'.format(type)) ) , dtype=object)
        data_pack = [ compounds, adjacencies, fps , proteins, inv_Temp, Temp, targets ]
    else:
        data_pack = [ compounds, adjacencies, fps , proteins, inv_Temp, Temp ]
        
    
    return data_pack

In [3]:
def split_data( data, ratio=0.1):
    '''
    Randomly split data into two datasets.
    '''
    idx = np.arange(len( data[0]))
    np.random.shuffle(idx)
    num_split = int(len(data[0]) * ratio)
    idx_1, idx_0 = idx[:num_split], idx[num_split:]
    data_0 = []
    data_1 = []
    for di in range(len(data)):
        if di != 3:
            data_0.append(data[di][idx_0])
            data_1.append(data[di][idx_1])
        if di == 3:
            tmp1 = []
            tmp2 = []
            idx_0 = list(idx_0)
            idx_1 = list(idx_1)
            for idx in idx_0:
                tmp1.append(data[di][idx])
            for idx in idx_1:
                tmp2.append(data[di][idx])
            data_0.append(tmp1)
            data_1.append(tmp2)
    return data_0, data_1

In [4]:
atom_dict = load_pickle(  '../data/dict/fingerprint_dict.pkl' )
word_dict = load_pickle(   '../data/dict/word_dict.pkl' )

datapack = load_esm_data(train_path, True, 'train')
test_data = load_esm_data(test_path, True, 'test')

train_data, dev_data = split_data( datapack, 0.1 )

num_epochs = int( args.num_epoch )#fixed value

In [5]:
class GATLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout=0.5, alpha=0.2, concat=True):
        super(GATLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2 * out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

    def forward(self, h, adj):
        Wh = torch.matmul(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = F.leaky_relu(torch.matmul(a_input, self.a).squeeze(3), self.alpha)

        zero_vec = -9e15 * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=2)
        # attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.bmm(attention, Wh)

        return F.elu(h_prime) if self.concat else h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        b = Wh.size()[0]
        N = Wh.size()[1]

        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=1)
        Wh_repeated_alternating = Wh.repeat_interleave(N, dim=0).view(b, N*N, self.out_features)
        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=2)

        return all_combinations_matrix.view(b, N, N, 2 * self.out_features)


class KcatModel(nn.Module):
    def __init__(self, n_atom, n_amino, comp_dim, prot_dim, gat_dim, num_head, dropout, alpha, window, layer_cnn, latent_dim, layer_out ):
        super(KcatModel, self).__init__()
        '''
        n_atom here stands for number of atom_features
        '''


        self.embedding_layer_atom = nn.Embedding(n_atom+1, comp_dim)
        self.embedding_layer_amino = nn.Embedding(n_amino+1, prot_dim)

        self.dropout = dropout
        self.alpha = alpha
        self.layer_cnn = layer_cnn
        self.latent_dim = latent_dim
        self.layer_out = layer_out

        self.gat_layers = [GATLayer(comp_dim, gat_dim, dropout=dropout, alpha=alpha, concat=True)
                           for _ in range(num_head)]
        for i, layer in enumerate(self.gat_layers):
            self.add_module('gat_layer_{}'.format(i), layer)
        self.gat_out = GATLayer(gat_dim * num_head, comp_dim, dropout=dropout, alpha=alpha, concat=False)
        self.W_comp = nn.Linear(comp_dim, latent_dim)

        self.conv_layers = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2*window+1,
                                                    stride=1, padding=window) for _ in range(layer_cnn)])
        self.W_prot = nn.Linear(prot_dim, latent_dim)

        self.fp0 = nn.Parameter(torch.empty(size=(1024, latent_dim)))
        nn.init.xavier_uniform_(self.fp0, gain=1.414)
        self.fp1 = nn.Parameter(torch.empty(size=(latent_dim, latent_dim)))
        nn.init.xavier_uniform_(self.fp1, gain=1.414)

        self.bidat_num = 4

        self.U = nn.ParameterList([nn.Parameter(torch.empty(size=(latent_dim, latent_dim))) for _ in range(self.bidat_num)])
        for i in range(self.bidat_num):
            nn.init.xavier_uniform_(self.U[i], gain=1.414)

        self.transform_c2p = nn.ModuleList([nn.Linear(latent_dim, latent_dim) for _ in range(self.bidat_num)])
        self.transform_p2c = nn.ModuleList([nn.Linear(latent_dim, latent_dim) for _ in range(self.bidat_num)])

        self.bihidden_c = nn.ModuleList([nn.Linear(latent_dim, latent_dim) for _ in range(self.bidat_num)])
        self.bihidden_p = nn.ModuleList([nn.Linear(latent_dim, latent_dim) for _ in range(self.bidat_num)])
        self.biatt_c = nn.ModuleList([nn.Linear(latent_dim * 2, 1) for _ in range(self.bidat_num)])
        self.biatt_p = nn.ModuleList([nn.Linear(latent_dim * 2, 1) for _ in range(self.bidat_num)])

        self.comb_c = nn.Linear(latent_dim * self.bidat_num, latent_dim)
        self.comb_p = nn.Linear(latent_dim * self.bidat_num, latent_dim)
   
        self.W_out = nn.ModuleList([nn.Linear(latent_dim * 3 + 2, latent_dim * 3 + 2)
                                    for _ in range(self.layer_out)])
    
        self.output = nn.Linear(latent_dim * 3 + 2, 1)
        

    def comp_gat(self, atoms, adj):
        atoms_vector = self.embedding_layer_atom(atoms)
        atoms_multi_head = torch.cat([gat(atoms_vector, adj) for gat in self.gat_layers], dim=2)
        atoms_vector = F.elu(self.gat_out(atoms_multi_head, adj))
        atoms_vector = F.leaky_relu(self.W_comp(atoms_vector), self.alpha)
        return atoms_vector #(B,L, 40)

    def prot_cnn(self, amino ):
        amino_vector = self.embedding_layer_amino(amino)
        amino_vector = torch.unsqueeze(amino_vector, 1)
        for i in range(self.layer_cnn):
            amino_vector = F.leaky_relu(self.conv_layers[i](amino_vector), self.alpha)
        amino_vector = torch.squeeze(amino_vector, 1)
        amino_vector = F.leaky_relu(self.W_prot(amino_vector), self.alpha)
        return amino_vector

    def mask_softmax(self, a, mask, dim=-1):
        a_max = torch.max(a, dim, keepdim=True)[0]
        a_exp = torch.exp(a - a_max)
        a_exp = a_exp * mask
        a_softmax = a_exp / (torch.sum(a_exp, dim, keepdim=True) + 1e-6)
        return a_softmax

    def bidirectional_attention_prediction(self,atoms_vector, atoms_mask, fps, amino_vector, amino_mask, inv_Temp, Temp):
        b = atoms_vector.shape[0]
        for i in range(self.bidat_num):
            A = torch.tanh(torch.matmul(torch.matmul(atoms_vector, self.U[i]), amino_vector.transpose(1, 2)))
            A = A * torch.matmul(atoms_mask.view(b, -1, 1), amino_mask.view(b, 1, -1))

            atoms_trans = torch.matmul(A, torch.tanh(self.transform_p2c[i](amino_vector)))
            amino_trans = torch.matmul(A.transpose(1, 2), torch.tanh(self.transform_c2p[i](atoms_vector)))

            atoms_tmp = torch.cat([torch.tanh(self.bihidden_c[i](atoms_vector)), atoms_trans], dim=2)
            amino_tmp = torch.cat([torch.tanh(self.bihidden_p[i](amino_vector)), amino_trans], dim=2)

            atoms_att = self.mask_softmax(self.biatt_c[i](atoms_tmp).view(b, -1), atoms_mask.view(b, -1))
            amino_att = self.mask_softmax(self.biatt_p[i](amino_tmp).view(b, -1), amino_mask.view(b, -1))

            cf = torch.sum(atoms_vector * atoms_att.view(b, -1, 1), dim=1)
            pf = torch.sum(amino_vector * amino_att.view(b, -1, 1), dim=1)

            if i == 0:
                cat_cf = cf
                cat_pf = pf
            else:
                cat_cf = torch.cat([cat_cf.view(b, -1), cf.view(b, -1)], dim=1)
                cat_pf = torch.cat([cat_pf.view(b, -1), pf.view(b, -1)], dim=1)

        inverse_Temp = inv_Temp.view(inv_Temp.shape[0],-1)
        Temperature = Temp.view(Temp.shape[0],-1)
        cf_final = torch.cat([self.comb_c(cat_cf).view(b, -1), fps.view(b, -1)], dim=1)#length = 2*d
        pf_final = self.comb_p(cat_pf)#length = d = 40
        cat_vector = torch.cat((cf_final, pf_final, inverse_Temp, Temperature), dim=1)#length=3*d+2
        
        for j in range(self.layer_out):
            cat_vector = F.leaky_relu(self.W_out[j](cat_vector), self.alpha )
            
        return self.output(cat_vector)

    def forward(self, atoms, atoms_mask, adjacency, amino_vector, amino_mask, fps, inv_Temp, Temp ):
        atoms_vector = self.comp_gat(atoms, adjacency)
        # amino_vector = self.prot_cnn( amino )

        super_feature = F.leaky_relu(torch.matmul(fps, self.fp0), 0.1)
        super_feature = F.leaky_relu(torch.matmul(super_feature, self.fp1), 0.1)

        prediction = self.bidirectional_attention_prediction( atoms_vector, atoms_mask, super_feature,\
                                                             amino_vector, amino_mask, inv_Temp, Temp )
        
        
        return prediction

In [6]:
def batch_pad(arr):
    '''
    Pad feature vectors all into the same length.
    '''
    N = max([a.shape[0] for a in arr])
    if arr[0].ndim == 1:
        new_arr = np.zeros((len(arr), N))
        new_arr_mask = np.zeros((len(arr), N))
        for i, a in enumerate(arr):
            n = a.shape[0]
            new_arr[i, :n] = a + 1
            new_arr_mask[i, :n] = 1
        return new_arr, new_arr_mask

    elif arr[0].ndim == 2:
        new_arr = np.zeros((len(arr), N, N))
        new_arr_mask = np.zeros((len(arr), N, N))
        for i, a in enumerate(arr):
            n = a.shape[0]
            new_arr[i, :n, :n] = a
            new_arr_mask[i, :n, :n] = 1
        return new_arr, new_arr_mask

def batch2tensor(batch_data, has_label, device):
    '''
    Convert loaded data into torch tensors.
    '''
    atoms_pad, atoms_mask = batch_pad(batch_data[0])
    adjacencies_pad, _ = batch_pad(batch_data[1])
    
    fps = batch_data[2]
    temp_arr = np.zeros((len(fps), 1024))
    for i,a in enumerate(fps):
        temp_arr[i, :] = np.array(list(a), dtype=int)
    fps = temp_arr
    
    amino_pad = batch_data[3][0]
    amino_mask = np.ones((1, len(amino_pad[0])))
    
    atoms_pad = Variable(torch.LongTensor(atoms_pad)).to(device)
    atoms_mask = Variable(torch.FloatTensor(atoms_mask)).to(device)
    adjacencies_pad = Variable(torch.LongTensor(adjacencies_pad)).to(device)
    fps = Variable(torch.FloatTensor(fps)).to(device)
    amino_pad = Variable(torch.Tensor(amino_pad)).to(device)
    amino_mask = Variable(torch.FloatTensor(amino_mask)).to(device)

    inv_Temp = batch_data[4]
    temp_arr = np.zeros((len(inv_Temp), 1))
    for i,a in enumerate( inv_Temp ):
        temp_arr[i, :] = a
    inv_Temp = torch.FloatTensor(temp_arr).to(device)
    
    Temp = batch_data[5]
    temp_arr = np.zeros((len(Temp), 1))
    for i,a in enumerate( Temp ):
        temp_arr[i, :] = a
    Temp = torch.FloatTensor(temp_arr).to(device)
    
    if has_label == False:
        return atoms_pad, atoms_mask, adjacencies_pad, fps, amino_pad, amino_mask, inv_Temp, Temp
    else:
        label = batch_data[6]
        temp_arr = np.zeros((len(label), 1))
        for i,a in enumerate( label ):
            temp_arr[i, :] = a
        label = torch.FloatTensor(temp_arr).to(device)

    return atoms_pad, atoms_mask, adjacencies_pad, fps, amino_pad, amino_mask, inv_Temp, Temp, label

In [13]:
class Alignment(nn.Module):
    def __init__(self, n_atom, n_amino, comp_dim, prot_dim, gat_dim, num_head, dropout, alpha, window, layer_cnn, latent_dim, layer_out ):
        super(Alignment, self).__init__()
        self.alpha = alpha
        latent_dim = 40
        self.W_a_1 = nn.Linear(1280, latent_dim) 
        self.W_a_2 = nn.Linear(latent_dim, latent_dim)
        self.W_a_3 = nn.Linear(latent_dim, latent_dim)

        self.output = nn.Linear(latent_dim, 1)

    def forward(self, amino_vector):
        #plan_3: different MLPs for different tensors
        # amino_vector = amino.unsqueeze(0)

        amino_vector = F.leaky_relu(self.W_a_1(amino_vector), self.alpha ) 
        amino_vector = F.leaky_relu(self.W_a_2(amino_vector), self.alpha )
        amino_vector = F.leaky_relu(self.W_a_3(amino_vector), self.alpha ) 

        size = amino_vector.size(1)
        return self.output(torch.mean(amino_vector[:, 1:size-1, :], dim=1)), amino_vector

def test(model, model_mlp, data_test, batch_size, device):
    model_mlp.eval()
    model.eval()
    predictions = []
    labels = []
    idx = [i for i in range(len(data_test[0]))]
    min_size = 1   #4
    
    for i in range(math.ceil(len(data_test[0]) / min_size)):
        batch_data = []
        for di in range(len(data_test)):
            if di != 3:
                batch_data.append(data_test[di][idx[ i* min_size: (i + 1) * min_size]])
            else:
                tmp = []
                for id in idx[ i* min_size: (i + 1) * min_size]:
                    tmp.append(data_test[di][id])
                batch_data.append(tmp)
                    
        # batch_data = [data_test[di][i * batch_size: (i + 1) * batch_size] for di in range(len(data_test))]
        atoms_pad, atoms_mask, adjacencies_pad, batch_fps, amino_pad,\
                            amino_mask, inv_Temp, Temp, label = batch2tensor(batch_data, True, device)
        with torch.no_grad():
            # pred, _ = model_mlp(amino_pad)
            pred, amino_pad = model_mlp(amino_pad)
            pred = model( atoms_pad, atoms_mask, adjacencies_pad, amino_pad, amino_mask, batch_fps, inv_Temp, Temp )
            
        predictions += pred.cpu().detach().numpy().reshape(-1).tolist()
        labels += label.cpu().numpy().reshape(-1).tolist()
        
    predictions = np.array(predictions)
    labels = np.array(labels)
    rmse, r2 = scores(labels, predictions)
    
    return rmse, r2

In [14]:
def train_eval(model, model_mlp, data_train, data_test, data_dev, device, lr, batch_size, lr_decay, decay_interval, num_epochs ):
    criterion = F.mse_loss
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0, amsgrad=True)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= decay_interval, gamma=lr_decay)
    idx = np.arange(len(data_train[0]))
    
    min_size = 1   #4
    if batch_size > min_size:
        div_min = int(batch_size / min_size)
    else:
        div_min = 1
        
    rmse_train_scores, r2_train_scores, rmse_test_scores, r2_test_scores, rmse_dev_scores, r2_dev_scores = [],[],[],[],[],[]
    for epoch in range(num_epochs):
             
        np.random.shuffle(idx)
        model.train()
        predictions = []
        labels = []
        for i in range(math.ceil( len(data_train[0]) / min_size )):
            batch_data = []
            for di in range(len(data_train)):
                if di != 3:
                    batch_data.append(data_train[di][idx[ i* min_size: (i + 1) * min_size]])
                else:
                    tmp = []
                    for id in idx[ i* min_size: (i + 1) * min_size]:
                        tmp.append(data_train[di][id])
                    batch_data.append(tmp)
            # batch_data = [data_train[di][idx[i]] \
            #               for di in range(len(data_train))]
            atoms_pad, atoms_mask, adjacencies_pad, batch_fps, amino_pad,\
                            amino_mask, inv_Temp, Temp, label = batch2tensor(batch_data, True, device)
            
            # pred, _ = model_mlp(amino_pad)
            pred, amino_pad = model_mlp(amino_pad)
            pred = model( atoms_pad, atoms_mask, adjacencies_pad, amino_pad, amino_mask, batch_fps, inv_Temp, Temp )
            loss = criterion(pred.float(), label.float())
            predictions += pred.cpu().detach().numpy().reshape(-1).tolist()
            labels += label.cpu().numpy().reshape(-1).tolist()
            loss.backward()
            if i % div_min == 0 and i != 0:    
                optimizer.step()
                optimizer.zero_grad()

        
        predictions = np.array(predictions)
        labels = np.array(labels)
        rmse_train, r2_train = scores( labels, predictions )
        rmse_dev, r2_dev = test( model, model_mlp, data_dev, batch_size, device ) #dev dataset
        rmse_test, r2_test = test(model, model_mlp, data_test, batch_size, device) # test dataset
        
        if rmse_test < 0.91:
            print('Best model found at epoch=' + str(epoch) + '!')
            best_model_pth = '../data/my_performance/model_latentdim=' + str(model.latent_dim) + '_outlayer=' + str(model.layer_out)
            best_model_pth = best_model_pth + '_rmsetest='+str( round(rmse_test,4) )+'_rmsedev='+str( round(rmse_dev,4) ) +'.pth'
            torch.save( model.state_dict(), best_model_pth)
            best_model_pth = '../data/my_performance/model_mlp_latentdim=' + str(model.latent_dim) + '_outlayer=' + str(model.layer_out)
            best_model_pth = best_model_pth + '_rmsetest='+str( round(rmse_test,4) )+'_rmsedev='+str( round(rmse_dev,4) ) +'.pth'
            torch.save( model_mlp.state_dict(), best_model_pth)


        rmse_train_scores.append( rmse_train )
        r2_train_scores.append( r2_train )
        rmse_dev_scores.append( rmse_dev )
        r2_dev_scores.append( r2_dev )
        rmse_test_scores.append( rmse_test )
        r2_test_scores.append( r2_test )

        
        print('epoch: '+str(epoch)+'/'+ str(num_epochs) +';  rmse test: ' + str(rmse_test) + '; r2 test: ' + str(r2_test) )
        

        scheduler.step()
        
    return rmse_train_scores, r2_train_scores, rmse_test_scores, r2_test_scores, rmse_dev_scores, r2_dev_scores 

In [15]:
model = KcatModel( len(atom_dict), len(word_dict), comp_dim, prot_dim, gat_dim, num_head, \
                          dropout, alpha, window, layer_cnn, latent_dim, layer_out )
model.to(device)
# model.load_state_dict(torch.load('../data/performances/model_latentdim=40_outlayer=4_rmsetest=0.8854_rmsedev=0.908.pth', map_location=device))


model_mlp = Alignment( len(atom_dict), len(word_dict), comp_dim, prot_dim, gat_dim, num_head, \
                          dropout, alpha, window, layer_cnn, latent_dim, layer_out )
model_mlp.to(device)
rmse_train_scores, r2_train_scores, rmse_test_scores, r2_test_scores ,rmse_dev_scores, r2_dev_scores = \
        train_eval(model, model_mlp , train_data, test_data, dev_data, device, lr, batch_size, lr_decay,\
                   decay_interval, num_epochs=1) #num_epochs=40


epoch: 0/1;  rmse test: 1.327613; r2 test: 0.247503


In [ ]:
epoch_inds = list(np.arange(1,num_epochs+1))

result = pd.DataFrame(zip( epoch_inds,rmse_train_scores, r2_train_scores, rmse_test_scores, r2_test_scores, \
                                rmse_dev_scores, r2_dev_scores ),\
                        columns = ['epoch','RMSE_train','R2_train','RMSE_test','R2_test','RMSE_dev','R2_dev'])

output_path = os.path.join(  '../data/my_performance/',os.path.basename(param_dict_pkl).split('.')[0] + \
                        '_lr=' + str(lr) + '_batch='+str(batch_size)+ \
                        '_lr_decay=' + str(lr_decay) + '_decay_interval=' + str(decay_interval) + \
                        '_shuffle_T=False'+'.csv' )

result.to_csv(output_path,index=None)
print('Done for ' + param_dict_pkl +'.')